In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check\
  torch==1.13.1\
  torchdata==0.5.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install \
  transformers==4.27.2 \
  datasets==2.11.0 --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Summarize Dialogue without Prompt Engineering

##### Use Case : Generation summary of a dialogue with pre-train LLM FLAN-T5 from HuggingFace. 

##### Loading dataset from Hugging Face DialogSum

In [6]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Extracting data files: 100%|█████████████████████| 3/3 [00:00<00:00, 804.95it/s]


Dataset csv downloaded and prepared to /Users/amiteshgangrade/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1222.95it/s]


In [14]:
example_indices = [40,200]

dash_line = '_'.join('' for x in range(100))
for i , index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print("Input Dialogue:")
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BaseLine Human Summary : ')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

___________________________________________________________________________________________________
Example  1
___________________________________________________________________________________________________
Input Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
BaseLine Human Summary : 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________

___________________________________________________________________________________________________
E

In [12]:
model_name = 'google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [26]:
sentence = "What time is it , Tom?"

sentence_encoded = tokenizer(sentence,return_tensors='pt')
print(sentence_encoded)

sentence_decoded = tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens = True
)
print("Encoded Sentence :")
print(sentence_encoded)
print('Decoded Sentence : ')
print(sentence_decoded)

{'input_ids': tensor([[ 363,   97,   19,   34,    3,    6, 3059,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Encoded Sentence :
{'input_ids': tensor([[ 363,   97,   19,   34,    3,    6, 3059,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Decoded Sentence : 
What time is it, Tom?


### Summarize Dialogue with an Instruction Prompt

###### Zero Shot Inference with an Instruction Prompt

In [27]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    prompt = f"""
    Summarize the following conversation.
    {dialogue}
    Summary:
    """
    inputs = tokenizer(dialogue,return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    
    print(dash_line)
    print("Example ", i+1)
    print(dash_line)
    print(f'INPUT Propmt:\n{dialogue}')
    print(dash_line)
    print(f'Baseline Human Summary :\n{summary}')
    print(dash_line)
    print(f'Model Generation - Without Prompt Engineering : \n{output}')
    

___________________________________________________________________________________________________
Example  1
___________________________________________________________________________________________________
INPUT Propmt:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
___________________________________________________________________________________________________
Baseline Human Summary :
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Model Generation - Without Prompt Engineering : 
Person1: It's ten to nine.
_____________________________

In [30]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    prompt = f"""
    Dialogue :
    {dialogue}
    What was going on?
    """
    
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
        inputs["input_ids"],
        max_new_tokens=True
    )[0],
    skip_special_tokens=True
    )
    
    
    print(dash_line)
    print("Example : ", i+1)
    print(dash_line)
    print(f'Input Prompt : \n{prompt}')
    print(dash_line)
    print(f'Baseline Human Summary : \n{summary}')
    print(dash_line)
    print(f"Model Generation - Zero Shot : \n{output}")

___________________________________________________________________________________________________
Example :  1
___________________________________________________________________________________________________
Input Prompt : 

    Dialogue :
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    What was going on?
    
___________________________________________________________________________________________________
Baseline Human Summary : 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
___________________________________________________________________________________________________
Model Generation - Zero Shot : 
Tom
________________

### Summarize Dialouge with One Shot and Few Shot Inference 

One Shot and Few Shot Inferences 

### One Shot Inference 

In [45]:
def make_prompt(example_indices_full, example_index_to_summarize):
    
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt += f"""
        Dialogue : 
        {dialogue}
        What was going on?
        {summary}
        """
        
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    prompt += f"""
    Dialogue : 
    {dialogue}
    What was going on ?
    """
        
    return prompt


In [46]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full,example_index_to_summarize)

print(one_shot_prompt)


        Dialogue : 
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
    Dialogue : 
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a f

In [49]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt,return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'Baseline Human Summary : \n{summary}')
print(dash_line)
print(f'Model Generation - One Shot Infenrence : \n{output}')

___________________________________________________________________________________________________
Baseline Human Summary : 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
Model Generation - One Shot Infenrence : 
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software.


### Few Shot Inference 

Few Shot Inference by adding two more full dialogue-summay pairs to the prompt

In [50]:
example_indices_full = [40,60,120]
example_index_to_summarize = 200

few_shot_propmt = make_prompt(example_indices_full,example_index_to_summarize)

print(few_shot_propmt)


        Dialogue : 
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue : 
        #Person1#: Hey, Frank. I heard you got a new job.
#Person2#: Yeah, Judy. I will be working for the Post Office. It's not a bad job.
#Person1#: Is it true that you have a heavy work schedule?
#Person2#: That's right. I am supposed to work at 5am everyday, and I only get 45 minutes for lunch.
#Person1#: So, why did you apply for such a demanding job?
#Person2#: Well, the government offers its employees excellent health insurance benefits.
#Person1#: Oh

In [51]:
summary = dataset['test'][example_index_to_summarize]['summary']

input = tokenizer(few_shot_propmt,return_tensors='pt')

output = tokenizer.decode(
    model.generate(
        input['input_ids'],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'Baseline Human Summary:\n{summary}')
print(dash_line)
print(f'Human Generation - Few Shot Inference : {output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (831 > 512). Running this sequence through the model will result in indexing errors


___________________________________________________________________________________________________
Baseline Human Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
___________________________________________________________________________________________________
Human Generation - Few Shot Inference : #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


Model Does not Perform that good with Few Shot learning as well , so it is better to fine tune the model or to use another model to work on the specific task.